In [60]:
import pandas as pd
input_file = '../data/BGL/BGL.log_structured.csv'
df = pd.read_csv(input_file)

df['Time'] = pd.to_datetime(df['Time'], format="%Y-%m-%d-%H.%M.%S.%f")



In [152]:
def is_anomaly(s):
        return len(list(filter(lambda x: x != '-', s))) > 0

In [154]:
g = df.groupby([pd.Grouper(key='Time', freq='6H'), 'EventId'])

g.first()


KeyError: 'Label'

In [143]:

def count_events(df):
        return df['EventId'].value_counts()
anomalies = []
event_counts = []
for group in g:
        anomaly = is_anomaly(group[1]['Label'])
        group[1]['anomaly'] = anomaly
        anomalies.append(anomaly)
        event_counts.append(count_events(group[1]).to_dict())

In [147]:
g.head()


,LineId,Label,Timestamp,Date,Node,Time,NodeRepeat,Type,Component,Level,Content,EventId,EventTemplate,ParameterList
0,1,-,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03 15:42:50.363779,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,instruction cache parity error corrected,3aa50e45,instruction cache parity error corrected,[]
1,2,-,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03 15:42:50.527847,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,instruction cache parity error corrected,3aa50e45,instruction cache parity error corrected,[]
2,3,-,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03 15:42:50.675872,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,instruction cache parity error corrected,3aa50e45,instruction cache parity error corrected,[]
3,4,-,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03 15:42:50.823719,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,instruction cache parity error corrected,3aa50e45,instruction cache parity error corrected,[]
4,5,-,1117838570,2005.06.03,R02-M1-N0-C:J12-U11,2005-06-03 15:42:50.982731,R02-M1-N0-C:J12-U11,RAS,KERNEL,INFO,instruction cache parity error corrected,3aa50e45,instruction cache parity error corrected,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4713485,4713486,KERNSOCK,1136390405,2006.01.04,R50-M1-NC-I:J18-U11,2006-01-04 08:00:05.142912,R50-M1-NC-I:J18-U11,RAS,KERNEL,FATAL,idoproxy communication failure: socket closed,d7623fbd,idoproxy communication failure: socket closed,[]
4713486,4713487,KERNSOCK,1136390405,2006.01.04,R31-M1-NC-I:J18-U11,2006-01-04 08:00:05.143845,R31-M1-NC-I:J18-U11,RAS,KERNEL,FATAL,idoproxy communication failure: socket closed,d7623fbd,idoproxy communication failure: socket closed,[]
4713487,4713488,KERNSOCK,1136390405,2006.01.04,R20-M1-NC-I:J18-U11,2006-01-04 08:00:05.144930,R20-M1-NC-I:J18-U11,RAS,KERNEL,FATAL,idoproxy communication failure: socket closed,d7623fbd,idoproxy communication failure: socket closed,[]
4713488,4713489,KERNSOCK,1136390405,2006.01.04,R00-M0-NC-I:J18-U11,2006-01-04 08:00:05.167045,R00-M0-NC-I:J18-U11,RAS,KERNEL,FATAL,idoproxy communication failure: socket closed,d7623fbd,idoproxy communication failure: socket closed,[]


In [148]:

g = g.apply(lambda x: x.shape[0]).reset_index()

g.head()
# g['anomaly'] = anomalies

AttributeError: 'DataFrameGroupBy' object has no attribute 'anomaly'

In [133]:
g

,Time,EventId,0
0,2005-06-03 12:00:00,070de4aa,4286
1,2005-06-03 12:00:00,147cfcff,19
2,2005-06-03 12:00:00,3aa50e45,4863
3,2005-06-03 12:00:00,4cecb415,5
4,2005-06-03 12:00:00,5569cffc,10
...,...,...,...
17516,2006-01-04 06:00:00,7a03d1e9,10
17517,2006-01-04 06:00:00,8df7ac9e,5
17518,2006-01-04 06:00:00,996c9c62,1
17519,2006-01-04 06:00:00,d7623fbd,8


In [134]:
event_count_vector = g.pivot(index="Time",
        columns="EventId").fillna(0)

In [135]:

event_count_vector

0                                               \
EventId             0045c3a8 01c4e373 02430fd4 0256fb0b 0315e48d 0328c31b   
Time                                                                        
2005-06-03 12:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2005-06-03 18:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2005-06-04 00:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2005-06-04 06:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2005-06-04 12:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
...                      ...      ...      ...      ...      ...      ...   
2006-01-03 06:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2006-01-03 12:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2006-01-03 18:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2006-01-04 00:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2006-01-04 06:00:00      0.0      0.0      0.0      0.0      0.0      0.0   

                                                         ...           \
EventId             045f554d 04639923 04f5a2ae 0509d32d  ... f7c382bd   
Time                                                     ...            
2005-06-03 12:00:00      0.0      0.0      0.0      0.0  ...      0.0   
2005-06-03 18:00:00      0.0      0.0      0.0      0.0  ...      0.0   
2005-06-04 00:00:00      0.0      0.0      0.0      0.0  ...      0.0   
2005-06-04 06:00:00      0.0      0.0      0.0      0.0  ...      0.0   
2005-06-04 12:00:00      0.0      0.0      0.0      0.0  ...      0.0   
...                      ...      ...      ...      ...  ...      ...   
2006-01-03 06:00:00      0.0      0.0      0.0      0.0  ...      0.0   
2006-01-03 12:00:00      0.0      0.0      0.0      0.0  ...      0.0   
2006-01-03 18:00:00      0.0      0.0      0.0      0.0  ...      0.0   
2006-01-04 00:00:00      0.0      0.0      0.0      0.0  ...      0.0   
2006-01-04 06:00:00      0.0      0.0      0.0      0.0  ...      0.0   

                                                                           \
EventId             f80a5ed5 f97e57f1 f9df8a24 fac52822 fb1560b4 fb225cd2   
Time                                                                        
2005-06-03 12:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2005-06-03 18:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2005-06-04 00:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2005-06-04 06:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2005-06-04 12:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
...                      ...      ...      ...      ...      ...      ...   
2006-01-03 06:00:00      0.0      0.0      3.0      0.0      0.0      0.0   
2006-01-03 12:00:00      0.0      0.0      1.0      0.0      0.0      0.0   
2006-01-03 18:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2006-01-04 00:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2006-01-04 06:00:00      0.0      0.0      0.0      0.0      0.0      0.0   

                                                
EventId             fe297487 feaae979 ff52c40c  
Time                                            
2005-06-03 12:00:00      0.0      0.0      0.0  
2005-06-03 18:00:00      0.0      0.0      0.0  
2005-06-04 00:00:00      0.0      0.0      0.0  
2005-06-04 06:00:00      0.0      0.0      0.0  
2005-06-04 12:00:00      0.0      0.0      0.0  
...                      ...      ...      ...  
2006-01-03 06:00:00      0.0      0.0      0.0  
2006-01-03 12:00:00      0.0      0.0      0.0  
2006-01-03 18:00:00      0.0      0.0      0.0  
2006-01-04 00:00:00      0.0      0.0      0.0  
2006-01-04 06:00:00      0.0      0.0      0.0  

[820 rows x 387 columns]

In [43]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(event_count_vector, test_size=0.2)


In [44]:
train   

index                                               \
EventId             0045c3a8 01c4e373 02430fd4 0256fb0b 0315e48d 0328c31b   
Time                                                                        
2005-10-22 00:00:00      0.0      0.0  10783.0      0.0      0.0  10784.0   
2005-10-15 00:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2005-07-05 12:00:00      0.0      0.0   2264.0      0.0      0.0   2265.0   
2005-10-18 18:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2005-07-12 18:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
...                      ...      ...      ...      ...      ...      ...   
2005-10-11 00:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2005-07-22 18:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2005-07-11 00:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2005-11-15 18:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2005-09-21 06:00:00      0.0      0.0   7974.0      0.0      0.0   7975.0   

                                                         ...        0  \
EventId             045f554d 04639923 04f5a2ae 0509d32d  ... f7c382bd   
Time                                                     ...            
2005-10-22 00:00:00  10785.0      0.0      0.0      0.0  ...      1.0   
2005-10-15 00:00:00      0.0      0.0      0.0      0.0  ...      0.0   
2005-07-05 12:00:00   2266.0      0.0      0.0      0.0  ...      0.0   
2005-10-18 18:00:00      0.0      0.0      0.0      0.0  ...      0.0   
2005-07-12 18:00:00      0.0      0.0      0.0      0.0  ...      0.0   
...                      ...      ...      ...      ...  ...      ...   
2005-10-11 00:00:00      0.0      0.0      0.0      0.0  ...      0.0   
2005-07-22 18:00:00      0.0      0.0      0.0      0.0  ...      0.0   
2005-07-11 00:00:00      0.0      0.0      0.0      0.0  ...      0.0   
2005-11-15 18:00:00      0.0      0.0      0.0      0.0  ...      0.0   
2005-09-21 06:00:00   7976.0      0.0      0.0      0.0  ...      1.0   

                                                                           \
EventId             f80a5ed5 f97e57f1 f9df8a24 fac52822 fb1560b4 fb225cd2   
Time                                                                        
2005-10-22 00:00:00      0.0      0.0      0.0      0.0      1.0      0.0   
2005-10-15 00:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2005-07-05 12:00:00      0.0      1.0      0.0      0.0      1.0      0.0   
2005-10-18 18:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2005-07-12 18:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
...                      ...      ...      ...      ...      ...      ...   
2005-10-11 00:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2005-07-22 18:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2005-07-11 00:00:00      0.0      0.0      0.0      0.0      0.0      0.0   
2005-11-15 18:00:00      0.0      0.0      1.0      0.0      0.0      0.0   
2005-09-21 06:00:00      0.0      0.0      0.0      0.0      1.0      0.0   

                                                
EventId             fe297487 feaae979 ff52c40c  
Time                                            
2005-10-22 00:00:00      0.0      0.0      0.0  
2005-10-15 00:00:00      0.0      0.0      0.0  
2005-07-05 12:00:00      0.0      0.0      0.0  
2005-10-18 18:00:00      0.0      0.0      0.0  
2005-07-12 18:00:00      0.0      0.0      0.0  
...                      ...      ...      ...  
2005-10-11 00:00:00      0.0      0.0      0.0  
2005-07-22 18:00:00      0.0      0.0      0.0  
2005-07-11 00:00:00      0.0      0.0      0.0  
2005-11-15 18:00:00      0.0      0.0      0.0  
2005-09-21 06:00:00      0.0      0.0      0.0  

[656 rows x 774 columns]